In [2]:
import pprint
import json
import pickle
from urllib.parse import unquote

import pandas as pd
import wikipedia
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from IPython.display import display

In [3]:
file = "../../data/processed/available_stocks/revolut.top50.2021-08-19T17:50:12.jsonl"
stocks = pd.read_json(file, lines=True)


display(stocks)
stocks = stocks.to_dict(orient="records")

name ticker                 sector  \
0                   Advanced Micro Devices    AMD  Electronic Technology   
1                                    Alcoa     AA    Non-Energy Minerals   
2                                  Alibaba   BABA           Retail Trade   
3                                    Ambev   ABEV  Consumer Non-Durables   
4                                      AMC    AMC      Consumer Services   
5                        American Airlines    AAL         Transportation   
6                                    Apple   AAPL  Electronic Technology   
7                                     AT&T      T         Communications   
8   Banco Bradesco S.A. – Preferred Shares    BBD                Finance   
9                          Bank of America    BAC                Finance   
10                                  BP plc     BP        Energy Minerals   
11                                Carnival    CCL      Consumer Services   
12                                 Chevron    CVX        Energy Minerals   
13                                   Cisco   CSCO    Technology Services   
14                               Citigroup      C                Finance   
15                   Cleveland-Cliffs Inc.    CLF    Non-Energy Minerals   
16                        DiDi Global Inc.   DIDI    Technology Services   
17                  Energy Transfer Equity     ET    Industrial Services   
18                             Exxon Mobil    XOM        Energy Minerals   
19                                    Ford      F      Consumer Durables   
20                                Freeport    FCX    Non-Energy Minerals   
21                          General Motors     GM      Consumer Durables   
22                                   Intel   INTC  Electronic Technology   
23                                   iQIYI     IQ    Technology Services   
24                           Itaú Unibanco   ITUB                Finance   
25                                  JD.com     JD           Retail Trade   
26                                  Kohl’s    KSS           Retail Trade   
27                                  Macy’s      M           Retail Trade   
28                       Micron Technology     MU  Electronic Technology   
29                               Microsoft   MSFT    Technology Services   
30                                 Moderna   MRNA      Health Technology   
31     New Oriental Education & Tech Group    EDU      Consumer Services   
32                                     NIO    NIO      Consumer Durables   
33          Norwegian Cruise Line Holdings   NCLH      Consumer Services   
34                                  Nvidia   NVDA  Electronic Technology   
35                              Occidental    OXY        Energy Minerals   
36                                Palantir   PLTR    Technology Services   
37                Petroleo Brasileiro S.A.    PBR        Energy Minerals   
38                                  Pfizer    PFE      Health Technology   
39                              Plug Power   PLUG  Electronic Technology   
40                            Schlumberger    SLB    Industrial Services   
41                       SoFi Technologies   SOFI                Finance   
42                    Taiwan Semiconductor    TSM  Electronic Technology   
43       Tencent Music Entertainment Group    TME    Technology Services   
44                                   Tesla   TSLA      Consumer Durables   
45                                    Uber   UBER    Technology Services   
46                     United States Steel      X    Non-Energy Minerals   
47                              UP Fintech   TIGR                Finance   
48                                    Vale   VALE    Non-Energy Minerals   
49                Vipshop Holdings Limited   VIPS           Retail Trade   

                           industry  
0                    Semiconductors  
1                          Aluminum  
2                   Internet Retail  
3        

In [4]:
wikis = {
    "Advanced Micro Devices": "https://en.wikipedia.org/wiki/Advanced_Micro_Devices",
    "Alcoa": "https://en.wikipedia.org/wiki/Alcoa",
    "Alibaba": "https://en.wikipedia.org/wiki/Alibaba_Group",
    "Ambev": "https://en.wikipedia.org/wiki/Ambev",
    "AMC": "https://en.wikipedia.org/wiki/AMC_Theatres",
    "American Airlines": "https://en.wikipedia.org/wiki/American_Airlines",
    "Apple": "https://en.wikipedia.org/wiki/Apple_Inc.",
    "AT&T": "https://en.wikipedia.org/wiki/AT%26T",
    "Banco Bradesco S.A. – Preferred Shares": "https://en.wikipedia.org/wiki/Banco_Bradesco",
    "Bank of America": "https://en.wikipedia.org/wiki/Bank_of_America",
    "BP plc": "https://en.wikipedia.org/wiki/BP",
    "Carnival": "https://en.wikipedia.org/wiki/Carnival_Corporation_%26_plc",
    "Chevron": "https://en.wikipedia.org/wiki/Chevron_Corporation",
    "Cisco": "https://en.wikipedia.org/wiki/Cisco_Systems",
    "Citigroup": "https://en.wikipedia.org/wiki/Citigroup",
    "Cleveland-Cliffs Inc.": "https://en.wikipedia.org/wiki/Cleveland-Cliffs",
    "DiDi Global Inc.": "https://en.wikipedia.org/wiki/DiDi",
    "Energy Transfer Equity": "https://en.wikipedia.org/wiki/Energy_Transfer_Partners",
    "Exxon Mobil": "https://en.wikipedia.org/wiki/ExxonMobil",
    "Ford": "https://en.wikipedia.org/wiki/Ford_Motor_Company",
    "Freeport": "https://en.wikipedia.org/wiki/Freeport-McMoRan",
    "General Motors": "https://en.wikipedia.org/wiki/General_Motors",
    "Intel": "https://en.wikipedia.org/wiki/Intel",
    "iQIYI": "https://en.wikipedia.org/wiki/IQIYI",
    "Itaú Unibanco": "https://en.wikipedia.org/wiki/Ita%C3%BA_Unibanco",
    "JD.com": "https://en.wikipedia.org/wiki/JD.com",
    "Kohl’s": "https://en.wikipedia.org/wiki/Kohl%27s",
    "Macy’s": "https://en.wikipedia.org/wiki/Macy%27s",
    "Micron Technology": "https://en.wikipedia.org/wiki/Micron_Technology",
    "Microsoft": "https://en.wikipedia.org/wiki/Microsoft",
    "Moderna": "https://en.wikipedia.org/wiki/Moderna",
    "New Oriental Education & Tech Group": "https://en.wikipedia.org/wiki/New_Oriental",
    "NIO": "https://en.wikipedia.org/wiki/NIO_(car_company)",
    "Norwegian Cruise Line Holdings": "https://en.wikipedia.org/wiki/Norwegian_Cruise_Line",
    "Nvidia": "https://en.wikipedia.org/wiki/Nvidia",
    "Occidental": "https://en.wikipedia.org/wiki/Occidental_Petroleum",
    "Palantir": "https://en.wikipedia.org/wiki/Palantir_Technologies",
    "Petroleo Brasileiro S.A.": "https://en.wikipedia.org/wiki/Petrobras",
    "Pfizer": "https://en.wikipedia.org/wiki/Pfizer",
    "Plug Power": "https://en.wikipedia.org/wiki/Plug_Power",
    "Schlumberger": "https://en.wikipedia.org/wiki/Schlumberger",
    "SoFi Technologies": "https://en.wikipedia.org/wiki/SoFi",
    "Taiwan Semiconductor": "https://en.wikipedia.org/wiki/TSMC",
    "Tencent Music Entertainment Group": "https://en.wikipedia.org/wiki/Tencent_Music",
    "Tesla": "https://en.wikipedia.org/wiki/Tesla,_Inc.",
    "Uber": "https://en.wikipedia.org/wiki/Uber",
    "United States Steel": "https://en.wikipedia.org/wiki/U.S._Steel",
    "UP Fintech": None,
    "Vale": "https://en.wikipedia.org/wiki/Vale_S.A.",
    "Vipshop Holdings Limited": "https://en.wikipedia.org/wiki/Vipshop"
}

In [5]:
pp = pprint.PrettyPrinter()

wikis_list = [v for k,v in wikis.items()]

for i, stock in enumerate(stocks):
    stock["wiki"] = wikis_list[i]
    
pp.pprint(stocks[:5])

[{'industry': 'Semiconductors',
  'name': 'Advanced Micro Devices',
  'sector': 'Electronic Technology',
  'ticker': 'AMD',
  'wiki': 'https://en.wikipedia.org/wiki/Advanced_Micro_Devices'},
 {'industry': 'Aluminum',
  'name': 'Alcoa',
  'sector': 'Non-Energy Minerals',
  'ticker': 'AA',
  'wiki': 'https://en.wikipedia.org/wiki/Alcoa'},
 {'industry': 'Internet Retail',
  'name': 'Alibaba',
  'sector': 'Retail Trade',
  'ticker': 'BABA',
  'wiki': 'https://en.wikipedia.org/wiki/Alibaba_Group'},
 {'industry': 'Beverages: Alcoholic',
  'name': 'Ambev',
  'sector': 'Consumer Non-Durables',
  'ticker': 'ABEV',
  'wiki': 'https://en.wikipedia.org/wiki/Ambev'},
 {'industry': 'Movies/Entertainment',
  'name': 'AMC',
  'sector': 'Consumer Services',
  'ticker': 'AMC',
  'wiki': 'https://en.wikipedia.org/wiki/AMC_Theatres'}]


In [6]:
def tokenize(text):
    stop_words = set(stopwords.words('english'))
    tokenized = [token.lower() for token in word_tokenize(text) if token.isalpha()]
    filtered = [token for token in tokenized if not token in stop_words]
    return filtered


def valid_paragraph(paragraph):
    return len(paragraph) > 0 and not paragraph.isspace() and not paragraph.startswith('==')


def get_plaintext(page_content):
    return ' '.join([paragraph for paragraph in page.content.split('\n') 
                     if valid_paragraph(paragraph)])

def get_url_endpoint(url):
    return url.split("/")[-1]

In [7]:
wikipedia.set_rate_limiting(True)

for stock in stocks:
    if stock["wiki"] is not None:
        
        # Read the main page
        title = get_url_endpoint(unquote(stock["wiki"]))
        page = wikipedia.WikipediaPage(title=title)
        
        stock["summary"] = tokenize(page.summary)
        stock["content"] = tokenize(get_plaintext(page.content))
        
        print(f"{title} has {len(page.links)} entities ...")
        stock["entities"] = []
        
        for entity in page.links:
            # Read the entity page
            try:
                page = wikipedia.WikipediaPage(title=entity)
            except wikipedia.DisambiguationError:
                print(f"Disambiguation error: {entity} ...")
            except wikipedia.PageError:
                print(f"Page error: {entity} ...")
            else:
                print(f"{title} -- {entity} ...")
                entity = dict(name=entity, 
                              summary=tokenize(page.summary))
#                               content=tokenize(get_plaintext(page.content)))
                stock["entities"].append(entity)
        
        print("\n")

Advanced_Micro_Devices has 851 entities ...
Advanced_Micro_Devices -- 16-bit ...
Advanced_Micro_Devices -- 22 nm ...
Advanced_Micro_Devices -- 32-bit ...
Advanced_Micro_Devices -- 32 nm ...
Advanced_Micro_Devices -- 3DNow! ...
Advanced_Micro_Devices -- 3D computer graphics ...
Advanced_Micro_Devices -- 3M ...
Advanced_Micro_Devices -- 3dfx Interactive ...
Advanced_Micro_Devices -- 4-bit ...
Advanced_Micro_Devices -- 45 nm ...
Advanced_Micro_Devices -- 45nm ...
Advanced_Micro_Devices -- 50x15 (AMD) ...
Advanced_Micro_Devices -- 50×15 (AMD) ...
Advanced_Micro_Devices -- 64-bit ...
Advanced_Micro_Devices -- 64-bit computing ...
Advanced_Micro_Devices -- 65nm ...
Advanced_Micro_Devices -- 7 nm process ...
Advanced_Micro_Devices -- 80186 ...
Advanced_Micro_Devices -- 80286 ...
Advanced_Micro_Devices -- 8086 ...
Advanced_Micro_Devices -- 90 nm ...
Advanced_Micro_Devices -- ABC News ...
Advanced_Micro_Devices -- ADP (company) ...
Advanced_Micro_Devices -- AES instruction set ...
Advanced_Micr

/home/lukas/anaconda3/envs/tldr/lib/python3.9/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/lukas/anaconda3/envs/tldr/lib/python3.9/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Disambiguation error: AMD (disambiguation) ...
Advanced_Micro_Devices -- AMD 10h ...
Advanced_Micro_Devices -- AMD 29000 ...
Advanced_Micro_Devices -- AMD 580 chipset series ...
Advanced_Micro_Devices -- AMD 690 chipset series ...
Advanced_Micro_Devices -- AMD 700 chipset series ...
Advanced_Micro_Devices -- AMD 800 chipset series ...
Advanced_Micro_Devices -- AMD APP SDK ...
Advanced_Micro_Devices -- AMD APU ...
Advanced_Micro_Devices -- AMD Accelerated Processing Unit ...
Advanced_Micro_Devices -- AMD Am2900 ...
Advanced_Micro_Devices -- AMD Am29000 ...
Advanced_Micro_Devices -- AMD Am9080 ...
Advanced_Micro_Devices -- AMD Catalyst ...
Advanced_Micro_Devices -- AMD CodeAnalyst ...
Advanced_Micro_Devices -- AMD CodeXL ...
Advanced_Micro_Devices -- AMD Core Math Library ...
Advanced_Micro_Devices -- AMD CrossFire ...
Advanced_Micro_Devices -- AMD CrossFireX ...
Advanced_Micro_Devices -- AMD Dragon ...
Advanced_Micro_Devices -- AMD Eyefinity ...
Advanced_Micro_Devices -- AMD FX ...
Adva

Advanced_Micro_Devices -- Color space ...
Advanced_Micro_Devices -- Comcast ...
Advanced_Micro_Devices -- Comic book ...
Advanced_Micro_Devices -- Commodore International ...
Advanced_Micro_Devices -- Compaq ...
Advanced_Micro_Devices -- Comparison of AMD chipsets ...
Advanced_Micro_Devices -- Complex programmable logic device ...
Advanced_Micro_Devices -- Computer architecture ...
Advanced_Micro_Devices -- Computer engineering ...
Advanced_Micro_Devices -- Computer manufacturer ...
Advanced_Micro_Devices -- Computer processor ...
Advanced_Micro_Devices -- Computex ...
Advanced_Micro_Devices -- Condé Nast ...
Advanced_Micro_Devices -- Conflict minerals ...
Advanced_Micro_Devices -- Consumer markets ...
Advanced_Micro_Devices -- Cool'n'Quiet ...
Advanced_Micro_Devices -- Copart ...
Advanced_Micro_Devices -- Core 2 ...
Advanced_Micro_Devices -- Coreboot ...
Advanced_Micro_Devices -- Corsair Gaming ...
Advanced_Micro_Devices -- Cortex-A57 ...
Advanced_Micro_Devices -- Costco ...
Advanced_

Advanced_Micro_Devices -- Interlink Electronics ...
Advanced_Micro_Devices -- Intuit ...
Advanced_Micro_Devices -- Intuitive Surgical ...
Advanced_Micro_Devices -- Irvine Company ...
Advanced_Micro_Devices -- JBL (company) ...
Advanced_Micro_Devices -- JD.com ...
Advanced_Micro_Devices -- JHDL ...
Advanced_Micro_Devices -- JOLT Digest ...
Advanced_Micro_Devices -- Jabil ...
Advanced_Micro_Devices -- Jack Gifford (businessman) ...
Advanced_Micro_Devices -- Jaguar (microarchitecture) ...
Advanced_Micro_Devices -- Java Optimized Processor ...
Advanced_Micro_Devices -- Jensen Electronics ...
Advanced_Micro_Devices -- Jerry Sanders (businessman) ...
Advanced_Micro_Devices -- Jim Keller (engineer) ...
Advanced_Micro_Devices -- John E. Caldwell ...
Advanced_Micro_Devices -- KEMET Corporation ...
Advanced_Micro_Devices -- KLA-Tencor ...
Advanced_Micro_Devices -- KLA Corporation ...
Advanced_Micro_Devices -- Kenmore (brand) ...
Advanced_Micro_Devices -- Keurig Dr Pepper ...
Advanced_Micro_Devic

...

Tesla,_Inc. -- XPO Logistics ...
Tesla,_Inc. -- Xcel Energy ...
Tesla,_Inc. -- Xilinx ...
Tesla,_Inc. -- Xoom (web hosting) ...
Tesla,_Inc. -- Yahoo! ...
Tesla,_Inc. -- Yahoo! Finance ...
Tesla,_Inc. -- Yamaha Motor Company ...
Tesla,_Inc. -- Yammer ...
Tesla,_Inc. -- Yellow Coach Manufacturing Company ...
Tesla,_Inc. -- Yellow Corporation ...
Tesla,_Inc. -- Yelp ...
Tesla,_Inc. -- Yishan Wong ...
Tesla,_Inc. -- YouTube ...
Tesla,_Inc. -- ZAP (motor company) ...
Tesla,_Inc. -- Zach Kirkhorn ...
Tesla,_Inc. -- Ze'ev Drori ...
Tesla,_Inc. -- Zero-emissions vehicle ...
Tesla,_Inc. -- Zero Motorcycles ...
Tesla,_Inc. -- Zimmer (automobile) ...
Tesla,_Inc. -- Zip2 ...
Tesla,_Inc. -- Zoom Video Communications ...
Tesla,_Inc. -- Zynga ...


Uber has 470 entities ...
Uber -- 2014 Sydney hostage crisis ...
Uber -- 2016 Kalamazoo shootings ...
Uber -- 2017 London Bridge attack ...
Uber -- 2019 Lyft and Uber drivers' strikes ...
Uber -- 2020 California Proposition 22 ...
Uber -- 99 (app) ...
Uber

Uber -- Naver Corporation ...
Uber -- Nelson Chai ...
Uber -- NetEase ...
Uber -- Net income ...
Uber -- Netflix ...
Uber -- New Jersey Department of Labor and Workforce Development ...
Uber -- New York Stock Exchange ...
Uber -- Non-disclosure agreement ...
Uber -- Non-profit ...
Uber -- OCLC (identifier) ...
Uber -- Obstruction of justice ...
Uber -- Offshore company ...
Uber -- Offshore investment ...
Uber -- Ola Cabs ...
Uber -- Open innovation ...
Uber -- Oracle Corporation ...
Uber -- OregonLive.com ...
Uber -- Otto (company) ...
Uber -- Overtime ...
Uber -- Package delivery ...
Uber -- PandoDaily ...
Uber -- Paradise Papers ...
Uber -- Pasporta Servo ...
Uber -- Pathao ...
Uber -- Patreon ...
Uber -- Peer-to-peer banking ...
Uber -- Peer-to-peer carsharing ...
Uber -- Peer-to-peer lending ...
Uber -- Peer-to-peer ridesharing ...
Uber -- Pet shipping ...
Uber -- Philadelphia ...
Uber -- Physical disability ...
Uber -- Pickle (app) ...
Uber -- Pittsburgh ...
Uber -- Pittsburgh Pos

U.S._Steel -- Congress of Industrial Organizations ...
U.S._Steel -- Consol Energy ...
U.S._Steel -- Constitution ...
U.S._Steel -- Convict leasing ...
U.S._Steel -- Corporate raid ...
U.S._Steel -- Corporation ...
U.S._Steel -- Court of Flags Resort ...
U.S._Steel -- Culture of Pittsburgh ...
U.S._Steel -- Cyanide ...
U.S._Steel -- David Burritt ...
U.S._Steel -- David L. Lawrence Convention Center ...
Page error: David M. Roderick ...
U.S._Steel -- David S. Sutherland ...
U.S._Steel -- Daytona Beach, Florida ...
U.S._Steel -- Dearborn, Michigan ...
U.S._Steel -- Delaware River ...
U.S._Steel -- Devra Davis ...
U.S._Steel -- Dick's Sporting Goods ...
U.S._Steel -- Disney's Contemporary Resort ...
U.S._Steel -- Disney's Polynesian Resort ...
U.S._Steel -- Dividend ...
U.S._Steel -- Doi (identifier) ...
U.S._Steel -- Dollar Bank ...
U.S._Steel -- Douglas Blackmon ...
U.S._Steel -- Dow Jones Industrial Average ...
U.S._Steel -- Downtown Pittsburgh ...
U.S._Steel -- Dravo Corporation ...


U.S._Steel -- Reconstruction Era ...
U.S._Steel -- Renda Broadcasting ...
U.S._Steel -- Republic Steel ...
U.S._Steel -- Respironics ...
U.S._Steel -- River Rouge, Michigan ...
U.S._Steel -- Rockwell International ...
U.S._Steel -- Roger Blough ...
U.S._Steel -- Ron Chernow ...
U.S._Steel -- Ronald Reagan ...
U.S._Steel -- Rue21 ...
U.S._Steel -- Rust belt ...
U.S._Steel -- S&P 400 ...
U.S._Steel -- S&P 500 ...
U.S._Steel -- Saint Michael the Archangel Catholic Church (Chicago) ...
U.S._Steel -- Sarris Candies ...
U.S._Steel -- Serbia ...
U.S._Steel -- Settlement (litigation) ...
U.S._Steel -- Severstal North America ...
U.S._Steel -- Slavery by Another Name ...
U.S._Steel -- Slovakia ...
U.S._Steel -- Smederevo ...
U.S._Steel -- South Chicago ...
U.S._Steel -- South Korea ...
U.S._Steel -- South Works ...
U.S._Steel -- Sports & Exhibition Authority of Pittsburgh and Allegheny County ...
U.S._Steel -- Sports in Pittsburgh ...
U.S._Steel -- Spreadshirt ...
U.S._Steel -- St. Petersburg, 

Vale_S.A. -- MRV Engenharia ...
Vale_S.A. -- MS Vale Brasil ...
Vale_S.A. -- Madrid ...
Vale_S.A. -- Magazine Luiza ...
Vale_S.A. -- Manganese ...
Vale_S.A. -- Manganese ore ...
Vale_S.A. -- Maranhão ...
Vale_S.A. -- Marfrig ...
Vale_S.A. -- Mariana dam disaster ...
Vale_S.A. -- Metal ...
Vale_S.A. -- Metalúrgica Gerdau ...
Vale_S.A. -- Michel Braudeau ...
Vale_S.A. -- Minas Gerais ...
Page error: Minerva Foods ...
Vale_S.A. -- Mining ...
Vale_S.A. -- Mining Weekly ...
Vale_S.A. -- Mitsui ...
Vale_S.A. -- Mozambique ...
Vale_S.A. -- Multinational corporation ...
Page error: Multiplan S.A. ...
Vale_S.A. -- NASA ...
Vale_S.A. -- Nacala Logistics Corridor ...
Vale_S.A. -- National Public Radio ...
Vale_S.A. -- Natura & Co ...
Vale_S.A. -- Net income ...
Vale_S.A. -- New Caledonia ...
Vale_S.A. -- New York City ...
Vale_S.A. -- New York Stock Exchange ...
Vale_S.A. -- New York Times ...
Vale_S.A. -- Newcrest Mining ...
Vale_S.A. -- Newmont ...
Vale_S.A. -- Nickel ...
Vale_S.A. -- Nitrogen 

In [8]:
output_dir = "../../data/temp/"
output_file = "revolut.top50.wiki.UPDT.jsonl"

with open(output_dir + output_file, "w", encoding="utf8") as fp:
    for item in stocks:
        json.dump(item, fp, ensure_ascii=False)
        fp.write("\n")

In [9]:
output_file_pkl = "revolut.top50.wiki.UPDT.pkl"

pickle.dump(stocks, open(output_dir + output_file_pkl, 'wb'))